# Computational Theory

**Ethan Conneely (G00393941)**


Imports used for countdown

`random` was used for generating random games of countdown for the solver to solve.

`itertools` was used for getting the permutations for all the possible combinations of operations on the numbers.


In [422]:
import random
import itertools

# Game Rules

Here I have defined the rules for a valid game of countdown it can be used to generate a random game to test the solver on it.

The rules for the game are as follows:

- The player may choose the number of large numbers (25,50,75,100) to be used in the game 0-4.
  - The game then picks the number of large number from that list there will not be duplicates
- The remaining number are picked from a selection of 20 number from 1-10 each number is double up in the selection


In [423]:
def generate_game(bigNumbers) -> tuple[list[int], int]:
    if bigNumbers > 4 or bigNumbers < 0:
        raise ValueError("Invalid bigNumbers")

    largeNumbers = random.sample([25, 50, 75, 100], bigNumbers)
    smallNumbers = random.sample(
        list(range(1, 11)) + list(range(1, 11)), 6 - bigNumbers
    )

    numbers = largeNumbers + smallNumbers

    return (numbers, random.randint(101, 999))

Here is a lookup for the operations that can be done, division is integer division as we cannot at any point in the game have a decimal number this helps optimize the runtime of the algorithm.


In [424]:
ops = {
    "+": lambda x, y: x + y,
    "-": lambda x, y: x - y,
    "*": lambda x, y: x * y,
    "/": lambda x, y: x // y,  # integer division
}

In [425]:
def recursive_solve_countdown(numbers, target, calculations=[]):
    solutions: list = []

    # iterate over all pairs of numbers
    for pair in itertools.combinations(numbers, 2):
        # Make a copy and remove the pair from the list of numbers this will be passed down when we recursively sovle again
        computedNumbers = numbers.copy()
        computedNumbers.remove(pair[0])
        computedNumbers.remove(pair[1])

        # the reason this is fine for division is that if you ever divide
        # by a number on the left thats smaller than the right it would always be a decimal as its smaller than 1
        a = max(pair)
        b = min(pair)

        # Run the computation on each operator while doing some checks on division
        for op, calculate in ops.items():
            if a == 0 or b == 0:
                continue  # skip zero

            if op == "/" or op == "\\":

                if a == 1 or b == 1:
                    continue  # skip redundent division

            if op == "*" and (a == 1 or b == 1):
                continue  # skip redundent multiplication

            if op == "/":
                if a % b != 0:
                    continue  # skip none whole number division

            # reference the global variable for keeping track of the number of calculations
            global computation_count
            computation_count += 1

            # run the calculation using the lamda
            val = calculate(a, b)

            calcs = [
                *calculations,
                (a, op, b, val),
            ]  # Keep tract of the calculations done so far

            if val == target:
                solutions.append(calcs)

            sols = recursive_solve_countdown([val, *computedNumbers], target, calcs)
            if len(sols) != 0:
                solutions = [*sols, *solutions]

    return solutions

In [426]:
def operations_to_expressions(expr, operations):
    left = operations.get(expr[0])
    if left is None:
        left = expr[0]
    else:
        del operations[expr[0]]
        left = operations_to_expressions(left, operations)

    right = operations.get(expr[2])
    if right is None:
        right = expr[2]
    else:
        del operations[expr[2]]
        right = operations_to_expressions(right, operations)

    return f"({left}{expr[1]}{right})"

`get_expression_lookup` converts the operations that were done to its computed value e.g

```
1+2=3
```

So it would store it in the dictionary as key `3` value `1,+,3` this is used for quick lookup later when recursivly descending the expression tree.


In [427]:
def get_expression_lookup(operations):
    expressions = {}
    for expr in operations:
        expressions[expr[3]] = (expr[0], expr[1], expr[2])
    return expressions

In [428]:
def print_unique_solutions(solutions):
    unique_expressions: dict[str, int] = {}

    solutions.sort(key=lambda x: len(x))

    for operations in solutions:

        expressions = get_expression_lookup(operations)

        expr_paren = operations_to_expressions(operations[-1], expressions)

        del expressions[operations[-1][3]]  # remove target expression

        # If we have expressions left over it mean some of the operations were not used so we can
        # skip this as the solution where we dont have it left over will also be a solution
        if len(expressions) != 0:
            continue

        # remove solutions that result in the same expression
        if unique_expressions.get(expr_paren):
            continue

        unique_expressions[expr_paren] = len(operations)

        for expr in operations:
            print(f"{expr[0]} {expr[1]} {expr[2]} = {expr[3]}")

        res = float(eval(expr_paren))

        print(expr_paren + " = " + str(res))
        print()

        target = operations[-1][3]
        # assert that the evaluation is correct helps catch error when making optimizations
        assert res == float(target)

    print("Found expressions: ", len(solutions))
    print("Unique expressions: ", len(unique_expressions))

## Fixed Game


In [429]:
numbers = [75, 50, 2, 3, 7, 1]
target = 103

computation_count = 0
solutions = recursive_solve_countdown(numbers, target)

print_unique_solutions(solutions)

print(f"Computations: {computation_count:,}")

50 * 2 = 100
100 + 3 = 103
((50*2)+3) = 103.0

3 + 1 = 4
7 * 4 = 28
75 + 28 = 103
(75+(7*(3+1))) = 103.0

50 - 3 = 47
75 * 2 = 150
150 - 47 = 103
((75*2)-(50-3)) = 103.0

50 / 2 = 25
75 + 3 = 78
78 + 25 = 103
((75+3)+(50/2)) = 103.0

50 / 2 = 25
25 + 3 = 28
75 + 28 = 103
(75+((50/2)+3)) = 103.0

50 / 2 = 25
75 + 25 = 100
100 + 3 = 103
((75+(50/2))+3) = 103.0

75 * 2 = 150
150 + 3 = 153
153 - 50 = 103
(((75*2)+3)-50) = 103.0

75 * 2 = 150
150 - 50 = 100
100 + 3 = 103
(((75*2)-50)+3) = 103.0

7 - 1 = 6
50 * 2 = 100
6 - 3 = 3
100 + 3 = 103
((50*2)+((7-1)-3)) = 103.0

7 - 1 = 6
50 * 2 = 100
100 - 3 = 97
97 + 6 = 103
(((50*2)-3)+(7-1)) = 103.0

7 - 1 = 6
50 * 2 = 100
100 + 6 = 106
106 - 3 = 103
(((50*2)+(7-1))-3) = 103.0

7 - 1 = 6
50 + 6 = 56
56 / 2 = 28
75 + 28 = 103
(75+((50+(7-1))/2)) = 103.0

3 - 1 = 2
7 * 2 = 14
14 * 2 = 28
75 + 28 = 103
(75+((7*(3-1))*2)) = 103.0

3 - 1 = 2
50 - 2 = 48
48 * 2 = 96
96 + 7 = 103
(((50-(3-1))*2)+7) = 103.0

3 - 1 = 2
2 * 2 = 4
7 * 4 = 28
75 + 28 = 103
(

## Single Solution


In [430]:
numbers = [25, 50, 5, 9, 3, 10]
target = 899

computation_count = 0

solutions = recursive_solve_countdown(numbers, target)

print_unique_solutions(solutions)

print(f"Computations: {computation_count:,}")

50 * 9 = 450
450 - 3 = 447
447 * 10 = 4470
4470 + 25 = 4495
4495 / 5 = 899
(((((50*9)-3)*10)+25)/5) = 899.0

Found expressions:  1
Unique expressions:  1
Computations: 1,367,373


## Random Game


In [431]:
(numbers, target) = generate_game(3)

print("numbers", numbers)
print("target", target)
print()

computation_count = 0
solutions = recursive_solve_countdown(numbers, target)
print_unique_solutions(solutions)
print(f"Computations: {computation_count:,}")

numbers [75, 50, 25, 3, 4, 5]
target 258

5 - 3 = 2
50 + 4 = 54
75 + 54 = 129
129 * 2 = 258
((75+(50+4))*(5-3)) = 258.0

5 - 3 = 2
75 + 4 = 79
79 + 50 = 129
129 * 2 = 258
(((75+4)+50)*(5-3)) = 258.0

5 - 3 = 2
75 + 50 = 125
125 + 4 = 129
129 * 2 = 258
(((75+50)+4)*(5-3)) = 258.0

5 - 3 = 2
75 + 2 = 77
77 * 4 = 308
308 - 50 = 258
(((75+(5-3))*4)-50) = 258.0

5 + 3 = 8
75 * 4 = 300
50 - 8 = 42
300 - 42 = 258
((75*4)-(50-(5+3))) = 258.0

5 + 3 = 8
75 * 4 = 300
300 - 50 = 250
250 + 8 = 258
(((75*4)-50)+(5+3)) = 258.0

5 + 3 = 8
75 * 4 = 300
300 + 8 = 308
308 - 50 = 258
(((75*4)+(5+3))-50) = 258.0

25 + 5 = 30
75 - 3 = 72
72 * 4 = 288
288 - 30 = 258
(((75-3)*4)-(25+5)) = 258.0

25 - 3 = 22
75 - 5 = 70
70 * 4 = 280
280 - 22 = 258
(((75-5)*4)-(25-3)) = 258.0

25 + 3 = 28
50 - 4 = 46
46 * 5 = 230
230 + 28 = 258
(((50-4)*5)+(25+3)) = 258.0

50 - 5 = 45
75 * 4 = 300
45 - 3 = 42
300 - 42 = 258
((75*4)-((50-5)-3)) = 258.0

50 - 5 = 45
75 * 4 = 300
300 + 3 = 303
303 - 45 = 258
(((75*4)+3)-(50-5)) =

# Comparison

When comparing my implementation to some online versions I found that my version removed more duplicate equations.

I compared the result to [dcode.fr](https://www.dcode.fr/countdown-numbers-game)

This is the game i used as a test

```shell
75, 50, 2, 3, 7
103
```

dcode.fr comes back with 15 equations found where as my implementation comes back with 13.

The reason for this is that dcode.fr does not remove duplicate equations if they are not in the same order. Here for example my implementation eliminates these into just 1 as they are the same expression in the end.

```shell
75 * 2 = 150
50 - 3 = 47
150 - 47 = 103
```

```shell
50 - 3 = 47
75 * 2 = 150
150 - 47 = 103
```

Both give this expression

`((75*2)-(50-3)) = 103.0`
